<h1><center>Fraud Detection System</center></h1>

## **Importing Libraries**

In [162]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from cryptography.fernet import Fernet

## **Generating Transaction Data with Numpy**

In [163]:
# Parameters
num_transactions = 40000   # 40000 transactions
fraud_ratio = 0.1  # 10% of the transactions are fraud

# Transaction Types
transaction_types = ['purchase', 'transfer', 'withdrawal', 'payment']

# Merchant Categories
merchant_categories = ['electronics', 'clothing', 'groceries', 'travel', 'entertainment']

In [164]:
presentday = datetime.now() # or presentday = datetime.today()

yesterday = presentday - timedelta(30)
print(yesterday)

2024-06-14 09:27:37.650365


In [165]:
# Generate Transaction Data
data = {
    'transaction_id': np.arange(num_transactions), # Arranges the num_trasnsactions in ascending order
    'user_id': np.random.randint(100, 1000, num_transactions), # Randomly generates user_id between 100 to 1000
    'transaction_amount': np.round(np.random.uniform(100, 10000, num_transactions), 2), # Randomly generates transaction_amount between 1000 to 10000 for corresponding user_id
    'transaction_time': pd.date_range(start = yesterday, periods=num_transactions, freq='T'), # Generates transaction data for the past 30 days
    'transaction_type': np.random.choice(transaction_types, num_transactions),
    'merchant_category': np.random.choice(merchant_categories, num_transactions),
    'is_fraud': np.random.choice([0, 1], num_transactions, p=[1-fraud_ratio, fraud_ratio]) # Randomly 0's and 1's are assigned to correspoding transactions where 1 denotes fraudulant transaction
}

In [166]:
# Create DataFrame
transaction_data = pd.DataFrame(data)

In [167]:
# Setting working directory
%cd C:\Users\sanja\Downloads\FraudDetectionSystem

[Errno 2] No such file or directory: 'C:UserssanjaDownloadsFraudDetectionSystem'
/content


In [168]:
# Save dataset to CSV file
transaction_data.to_csv('transaction_data.csv', index=False)

In [169]:
# Display transaction data
transaction_data

,transaction_id,user_id,transaction_amount,transaction_time,transaction_type,merchant_category,is_fraud
0,0,261,5512.67,2024-06-14 09:27:37.650365,payment,entertainment,0
1,1,366,169.30,2024-06-14 09:28:37.650365,payment,clothing,0
2,2,474,4807.59,2024-06-14 09:29:37.650365,transfer,groceries,0
3,3,157,2039.05,2024-06-14 09:30:37.650365,purchase,entertainment,0
4,4,344,560.75,2024-06-14 09:31:37.650365,withdrawal,entertainment,0
...,...,...,...,...,...,...,...
39995,39995,548,408.82,2024-07-12 04:02:37.650365,purchase,travel,0
39996,39996,881,554.19,2024-07-12 04:03:37.650365,payment,groceries,0
39997,39997,378,7616.34,2024-07-12 04:04:37.650365,purchase,electronics,0
39998,39998,720,5290.33,2024-07-12 04:05:37.650365,payment,travel,0


## **Generate Encryption Key**

In [170]:
key = Fernet.generate_key()
cipher = Fernet(key)

## **Feature Engineering**

In [171]:
# Load the dataset
transaction_data = pd.read_csv('transaction_data.csv')

In [172]:
# Display transaction data
transaction_data

,transaction_id,user_id,transaction_amount,transaction_time,transaction_type,merchant_category,is_fraud
0,0,261,5512.67,2024-06-14 09:27:37.650365,payment,entertainment,0
1,1,366,169.30,2024-06-14 09:28:37.650365,payment,clothing,0
2,2,474,4807.59,2024-06-14 09:29:37.650365,transfer,groceries,0
3,3,157,2039.05,2024-06-14 09:30:37.650365,purchase,entertainment,0
4,4,344,560.75,2024-06-14 09:31:37.650365,withdrawal,entertainment,0
...,...,...,...,...,...,...,...
39995,39995,548,408.82,2024-07-12 04:02:37.650365,purchase,travel,0
39996,39996,881,554.19,2024-07-12 04:03:37.650365,payment,groceries,0
39997,39997,378,7616.34,2024-07-12 04:04:37.650365,purchase,electronics,0
39998,39998,720,5290.33,2024-07-12 04:05:37.650365,payment,travel,0


In [173]:
# Converts transaction_time to datetime format & extracts hour and day of the week from the transaction time
transaction_data['transaction_time'] = pd.to_datetime(transaction_data['transaction_time'])
transaction_data['transaction_hour'] = transaction_data['transaction_time'].dt.hour
transaction_data['transaction_day'] = transaction_data['transaction_time'].dt.dayofweek

In [174]:
# Display transaction data
transaction_data

,transaction_id,user_id,transaction_amount,transaction_time,transaction_type,merchant_category,is_fraud,transaction_hour,transaction_day
0,0,261,5512.67,2024-06-14 09:27:37.650365,payment,entertainment,0,9,4
1,1,366,169.30,2024-06-14 09:28:37.650365,payment,clothing,0,9,4
2,2,474,4807.59,2024-06-14 09:29:37.650365,transfer,groceries,0,9,4
3,3,157,2039.05,2024-06-14 09:30:37.650365,purchase,entertainment,0,9,4
4,4,344,560.75,2024-06-14 09:31:37.650365,withdrawal,entertainment,0,9,4
...,...,...,...,...,...,...,...,...,...
39995,39995,548,408.82,2024-07-12 04:02:37.650365,purchase,travel,0,4,4
39996,39996,881,554.19,2024-07-12 04:03:37.650365,payment,groceries,0,4,4
39997,39997,378,7616.34,2024-07-12 04:04:37.650365,purchase,electronics,0,4,4
39998,39998,720,5290.33,2024-07-12 04:05:37.650365,payment,travel,0,4,4


## **Data Encryption**

In [175]:
# Encrypts the transaction_id and transaction_time columns
transaction_data['transaction_id'] = transaction_data['transaction_id'].apply(lambda x: cipher.encrypt(str(x).encode()).decode())
transaction_data['transaction_time'] = transaction_data['transaction_time'].apply(lambda x: cipher.encrypt(str(x).encode()).decode())
transaction_data['user_id'] = transaction_data['user_id'].apply(lambda x: cipher.encrypt(str(x).encode()).decode())

# Saves the encryption key into file
with open('encryption_key.key', 'wb') as key_file:
    key_file.write(key)

In [176]:
# Display transaction data
transaction_data

,transaction_id,user_id,transaction_amount,transaction_time,transaction_type,merchant_category,is_fraud,transaction_hour,transaction_day
0,gAAAAABmk5oK1i-NBqUtH_9TsxDWuxJ3mcJjeeHJ3F6kOg...,gAAAAABmk5oQGSieOVgkoblFZK-6qHyAkL7k2-EFkrFnLd...,5512.67,gAAAAABmk5oNbucI5CqQmUrcWGqqMuWEOWuryFmlKV54z1...,payment,entertainment,0,9,4
1,gAAAAABmk5oKdZ6wS_e7sx62SgJBKUG1uop2U8nOqONtIT...,gAAAAABmk5oQ8ijbT3bbmSzuKo9X0cACkqlGOUNNvwt6Qc...,169.30,gAAAAABmk5oNrFEWuYqlVHt56r1Ez-hjtyueTWDne2qGvl...,payment,clothing,0,9,4
2,gAAAAABmk5oKzd0cZr5Aj9pxASLiXTJzAU-sLYU82xPs-m...,gAAAAABmk5oQH_Lq4QqviPQILFlJ3bilDlDMmIJw0Eg4il...,4807.59,gAAAAABmk5oNElTRWCXfxLscog0JCQqFV-B2lnaC7fvetV...,transfer,groceries,0,9,4
3,gAAAAABmk5oK34FOdP9JXPYpW2tt0LrbQQrGayatvl9SY4...,gAAAAABmk5oQhHDRB4kTgOJsJrHeTO-aobaGYFFYuMFgpH...,2039.05,gAAAAABmk5oNMqHCJLo5SDlAiAh-ETDlJzOXGea35wYQHL...,purchase,entertainment,0,9,4
4,gAAAAABmk5oKMlQixsfZJowP2HX9jN39fGBTc24ACNiKH1...,gAAAAABmk5oQQbpkDHj6A3TX8NipVpYRr-ZO5OrUlsny8h...,560.75,gAAAAABmk5oNwIfS4P3FjSGgAigDDXfNCjWC_OKL_CEOY_...,withdrawal,entertainment,0,9,4
...,...,...,...,...,...,...,...,...,...
39995,gAAAAABmk5oN6v8v2ngjbYxP39TtUAZs6vmNAMoOUkdfiR...,gAAAAABmk5oT9QyXjo5EIyWFseV1EUiCG_hAykRp_wcN1D...,408.82,gAAAAABmk5oQnSiWdjhwi2xLBlCpVPU4KqUMBz2mfPPobQ...,purchase,travel,0,4,4
39996,gAAAAABmk5oNiK0I_1oHSIobMfwv9fvb15AMBwBTEO4yNe...,gAAAAABmk5oTx-ScVSJcpyEDII318BRXhrL2TF8MS1lgwf...,554.19,gAAAAABmk5oQGTrpGeIOoghPisDNMyS9H3-KrGmsXgPMAI...,payment,groceries,0,4,4
39997,gAAAAABmk5oNtfeSoxKG0yWSJcHOiGMJ3fRPRMKCCgk79b...,gAAAAABmk5oTt67CHqAGv6lo6SqLocFJ6enrHX8wcu_e59...,7616.34,gAAAAABmk5oQERyxofuNZZbwSs7iAZ4w2lsio7MffCH00e...,purchase,electronics,0,4,4
39998,gAAAAABmk5oNXRufcJR69nsw3DR5_tIrOXCDDGWJx1Qyvy...,gAAAAABmk5oTns0FMiZSnvTfuWC_Z5FLQcgzNorNWq-g9D...,5290.33,gAAAAABmk5oQeqbdGVJ6C3g87GXIMNJWM5etU4q5UUUrNQ...,payment,travel,0,4,4


## **Data Anonymization**

In [177]:
# Additional Feature: According to the business needs, the Sensitive Data can also be dropped to make the user's transaction untraceable
# transaction_data = transaction_data.drop(columns=['user_id']

## **Differential Privacy**

In [178]:
# creates numerical and categorical features
numerical_features = ['transaction_amount']
categorical_features = ['transaction_type', 'merchant_category']

In [179]:
# Add_noise function - Adds noise to the numerical features to ensure differential privacy
def add_noise(data, epsilon=0.1):
    noise = np.random.laplace(0, 1/epsilon, data.shape)
    return data + noise

transaction_data[numerical_features] = add_noise(transaction_data[numerical_features])

## **Encode Categorical Features**

In [180]:
# Creates a preprocessing pipeline to scale numerical features and one-hot encode categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [181]:
# Apply the preprocessing pipeline.

# Drops 3 columns from transaction data and rest of the columns are assigned to X variable
x = transaction_data.drop(columns=['transaction_id', 'transaction_time', 'is_fraud'])

# Y vairable is assigned 'is_fraud' column
y = transaction_data['is_fraud']

# Preprocess the features of X and
x_preprocessed = preprocessor.fit_transform(x)

In [182]:
# Convert the preprocessed data back to a DataFrame for better visualization
x_preprocessed_df = pd.DataFrame(x_preprocessed, columns=preprocessor.get_feature_names_out())

# Converts the preprocessed data back to a DataFrame and adds transaction_hour and transaction_day
x_preprocessed_df['transaction_hour'] = transaction_data['transaction_hour']
x_preprocessed_df['transaction_day'] = transaction_data['transaction_day']

## **Data Splitting**

In [183]:
# Dataset is split into 2 sets: Training and testing. Data is split in the ratio of 80:20, where 20% of the data is used for testing the model's evaluation.
x_train, x_test, y_train, y_test = train_test_split(x_preprocessed_df, y, test_size=0.2)

## **Anomaly Detection using Isolation Forest Algorithm**

In [184]:
iso_forest = IsolationForest(contamination=0.05)
iso_forest.fit(x_train)

train_anomalies = iso_forest.predict(x_train)
test_anomalies = iso_forest.predict(x_test)

train_anomalies = np.where(train_anomalies == -1, 1, 0)
test_anomalies = np.where(test_anomalies == -1, 1, 0)

x_train['anomaly'] = train_anomalies
x_test['anomaly'] = test_anomalies

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


## **Pattern Recognition using Neural Network**

In [185]:
model = Sequential()
model.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [186]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [187]:
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/20
1000/1000 [==============================] - 3s 2ms/step - loss: 0.4147 - accuracy: 0.8833 - val_loss: 0.3510 - val_accuracy: 0.8999
Epoch 2/20
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3500 - accuracy: 0.8998 - val_loss: 0.3289 - val_accuracy: 0.8999
Epoch 3/20
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3375 - accuracy: 0.8999 - val_loss: 0.3268 - val_accuracy: 0.8999
Epoch 4/20
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3333 - accuracy: 0.8999 - val_loss: 0.3260 - val_accuracy: 0.8999
Epoch 5/20
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3299 - accuracy: 0.8999 - val_loss: 0.3261 - val_accuracy: 0.8999
Epoch 6/20
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3285 - accuracy: 0.8999 - val_loss: 0.3260 - val_accuracy: 0.8999
Epoch 7/20
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3275 - accuracy: 0.8999 - val_loss: 0.3265 - val_accuracy:

In [188]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {accuracy}')

250/250 [==============================] - 0s 1ms/step - loss: 0.3258 - accuracy: 0.8999
Test Accuracy: 0.8998749852180481


In [189]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int)

250/250 [==============================] - 0s 1ms/step


## **Model Evaluation**

In [190]:
from sklearn.metrics import classification_report, confusion_matrix
from simple_colors import *

print(blue("Confusion matrix:", ["bold"]))
print(confusion_matrix(y_test, y_pred))

print(blue('\nClassification Reports:', ["bold"]))
print(classification_report(y_test, y_pred))

Confusion matrix:
[[7199    0]
 [ 801    0]]

Classification Reports:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      7199
           1       0.00      0.00      0.00       801

    accuracy                           0.90      8000
   macro avg       0.45      0.50      0.47      8000
weighted avg       0.81      0.90      0.85      8000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Displaying Enrcypted Transaction Data**

In [191]:
transaction_data

,transaction_id,user_id,transaction_amount,transaction_time,transaction_type,merchant_category,is_fraud,transaction_hour,transaction_day
0,gAAAAABmk5oK1i-NBqUtH_9TsxDWuxJ3mcJjeeHJ3F6kOg...,gAAAAABmk5oQGSieOVgkoblFZK-6qHyAkL7k2-EFkrFnLd...,5513.098447,gAAAAABmk5oNbucI5CqQmUrcWGqqMuWEOWuryFmlKV54z1...,payment,entertainment,0,9,4
1,gAAAAABmk5oKdZ6wS_e7sx62SgJBKUG1uop2U8nOqONtIT...,gAAAAABmk5oQ8ijbT3bbmSzuKo9X0cACkqlGOUNNvwt6Qc...,164.192435,gAAAAABmk5oNrFEWuYqlVHt56r1Ez-hjtyueTWDne2qGvl...,payment,clothing,0,9,4
2,gAAAAABmk5oKzd0cZr5Aj9pxASLiXTJzAU-sLYU82xPs-m...,gAAAAABmk5oQH_Lq4QqviPQILFlJ3bilDlDMmIJw0Eg4il...,4811.747302,gAAAAABmk5oNElTRWCXfxLscog0JCQqFV-B2lnaC7fvetV...,transfer,groceries,0,9,4
3,gAAAAABmk5oK34FOdP9JXPYpW2tt0LrbQQrGayatvl9SY4...,gAAAAABmk5oQhHDRB4kTgOJsJrHeTO-aobaGYFFYuMFgpH...,2042.545395,gAAAAABmk5oNMqHCJLo5SDlAiAh-ETDlJzOXGea35wYQHL...,purchase,entertainment,0,9,4
4,gAAAAABmk5oKMlQixsfZJowP2HX9jN39fGBTc24ACNiKH1...,gAAAAABmk5oQQbpkDHj6A3TX8NipVpYRr-ZO5OrUlsny8h...,564.489986,gAAAAABmk5oNwIfS4P3FjSGgAigDDXfNCjWC_OKL_CEOY_...,withdrawal,entertainment,0,9,4
...,...,...,...,...,...,...,...,...,...
39995,gAAAAABmk5oN6v8v2ngjbYxP39TtUAZs6vmNAMoOUkdfiR...,gAAAAABmk5oT9QyXjo5EIyWFseV1EUiCG_hAykRp_wcN1D...,409.489488,gAAAAABmk5oQnSiWdjhwi2xLBlCpVPU4KqUMBz2mfPPobQ...,purchase,travel,0,4,4
39996,gAAAAABmk5oNiK0I_1oHSIobMfwv9fvb15AMBwBTEO4yNe...,gAAAAABmk5oTx-ScVSJcpyEDII318BRXhrL2TF8MS1lgwf...,559.030133,gAAAAABmk5oQGTrpGeIOoghPisDNMyS9H3-KrGmsXgPMAI...,payment,groceries,0,4,4
39997,gAAAAABmk5oNtfeSoxKG0yWSJcHOiGMJ3fRPRMKCCgk79b...,gAAAAABmk5oTt67CHqAGv6lo6SqLocFJ6enrHX8wcu_e59...,7615.455341,gAAAAABmk5oQERyxofuNZZbwSs7iAZ4w2lsio7MffCH00e...,purchase,electronics,0,4,4
39998,gAAAAABmk5oNXRufcJR69nsw3DR5_tIrOXCDDGWJx1Qyvy...,gAAAAABmk5oTns0FMiZSnvTfuWC_Z5FLQcgzNorNWq-g9D...,5285.546597,gAAAAABmk5oQeqbdGVJ6C3g87GXIMNJWM5etU4q5UUUrNQ...,payment,travel,0,4,4


In [192]:
# Check for present working directory
%pwd

'/content'

In [193]:
# Saves the encrypted data into a CSV file
transaction_data.to_csv('encrypted_transaction_data.csv', index=False)

## **Decrypting the encrypted PII**

In [194]:
transaction_data['transaction_id'] = transaction_data['transaction_id'].apply(lambda x: cipher.decrypt(x.encode()).decode())
transaction_data['transaction_time'] = transaction_data['transaction_time'].apply(lambda x: cipher.decrypt(x.encode()).decode())
transaction_data['user_id'] = transaction_data['user_id'].apply(lambda x: cipher.decrypt(x.encode()).decode())

In [195]:
# Convert transaction_time back to datetime
transaction_data['transaction_time'] = pd.to_datetime(transaction_data['transaction_time'])

In [196]:
transaction_data

,transaction_id,user_id,transaction_amount,transaction_time,transaction_type,merchant_category,is_fraud,transaction_hour,transaction_day
0,0,261,5513.098447,2024-06-14 09:27:37.650365,payment,entertainment,0,9,4
1,1,366,164.192435,2024-06-14 09:28:37.650365,payment,clothing,0,9,4
2,2,474,4811.747302,2024-06-14 09:29:37.650365,transfer,groceries,0,9,4
3,3,157,2042.545395,2024-06-14 09:30:37.650365,purchase,entertainment,0,9,4
4,4,344,564.489986,2024-06-14 09:31:37.650365,withdrawal,entertainment,0,9,4
...,...,...,...,...,...,...,...,...,...
39995,39995,548,409.489488,2024-07-12 04:02:37.650365,purchase,travel,0,4,4
39996,39996,881,559.030133,2024-07-12 04:03:37.650365,payment,groceries,0,4,4
39997,39997,378,7615.455341,2024-07-12 04:04:37.650365,purchase,electronics,0,4,4
39998,39998,720,5285.546597,2024-07-12 04:05:37.650365,payment,travel,0,4,4


In [197]:
# Check for present working directory
%pwd

'/content'

In [198]:
# If condition used to drop the columns added during feature engine
if 'transaction_hour' in transaction_data.columns:
    transaction_data = transaction_data.drop(columns=['transaction_hour'])

if 'transaction_day' in transaction_data.columns:
    transaction_data = transaction_data.drop(columns=['transaction_day'])

In [199]:
# Saves the decrypted data into a CSV file
transaction_data.to_csv('decrypted_transaction_data.csv', index=False)